In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!nvidia-smi

Sat Feb 10 00:54:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080        On  | 00000000:01:00.0  On |                  N/A |
| 32%   33C    P8              19W / 215W |    405MiB /  8192MiB |      3%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
import os
os.getcwd()

'/home/linh/projects/test_notebooks'

In [5]:
os.chdir('../IConNet')

In [6]:
import numpy as np
from tqdm import tqdm
import itertools
import matplotlib.pyplot as plt
import pandas as pd

# Audio
import librosa
import librosa.display

# Scikit learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.utils import class_weight

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [7]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
import gc
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import sys
import IPython.display as ipd
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [8]:
dataset = 'text_emotion'
input_feature = 'signals_3channels_win5stride1'
output_label = 'labels_13emotions'
data_dir = f'../data/nlp/{dataset}/'

In [9]:
data_path_prefix = f'{data_dir}preprocessed/{dataset}'
x_train = np.load(f'{data_path_prefix}.{input_feature}.train.npy', allow_pickle=True)
x_test = np.load(f'{data_path_prefix}.{input_feature}.test.npy', allow_pickle=True)
labels = np.load(f'{data_path_prefix}.classnames.npy', allow_pickle=True)
y_train = np.load(f'{data_path_prefix}.{output_label}.test.npy', allow_pickle=True)
y_test = np.load(f'{data_path_prefix}.{output_label}.test.npy', allow_pickle=True)

In [10]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape, labels, sep=' ')

(8000, 3, 1024) (8000, 3, 1024) (8000,) (8000,) ['anger' 'boredom' 'empty' 'enthusiasm' 'fun' 'happiness' 'hate' 'love'
 'neutral' 'relief' 'sadness' 'surprise' 'worry']


In [11]:
from torch.utils.data import TensorDataset, DataLoader

def create_data_loader(x_train, y_train, 
                       x_test, y_test, batch_size=32):
    x_train = torch.tensor(x_train, dtype=torch.float)
    y_train = torch.tensor(y_train)
    x_test = torch.tensor(x_test, dtype=torch.float)
    y_test = torch.tensor(y_test)
    train_data = TensorDataset(x_train, y_train)
    test_data = TensorDataset(x_test, y_test)
    
    train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
    test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)
    return train_loader, test_loader

In [12]:
batch_size = 128
train_loader, test_loader = create_data_loader(x_train, y_train, 
                       x_test, y_test, batch_size=batch_size)

In [13]:
from IConNet.nn.model import M11

In [14]:
from omegaconf import OmegaConf as ocf

In [66]:
del model
gc.collect()

1133

In [67]:
model_config_path = 'config/model/m10.yaml'
model_config = ocf.load(model_config_path)
model_config

{'name': 'M10', 'description': 'gated residual FirConv', 'fe': {'n_block': 2, 'n_channel': [128, 128], 'kernel_size': [511, 127], 'stride': [2, 8], 'window_k': [2, 9], 'residual_connection_type': 'concat', 'pooling': 'None'}, 'cls': {'n_block': 2, 'n_hidden_dim': [256, 256]}}

In [68]:
labels

array(['anger', 'boredom', 'empty', 'enthusiasm', 'fun', 'happiness',
       'hate', 'love', 'neutral', 'relief', 'sadness', 'surprise',
       'worry'], dtype=object)

In [69]:
model = M11(config=model_config,
           n_input=3, n_output=len(labels))
model

M11(
  (fe_blocks): FeBlocks(
    (blocks): ModuleList(
      (0-1): 2 x Sequential(
        (layer): FirConvLayer()
        (norm): CustomNormLayer(
          (layer): LocalResponseNorm(2, alpha=0.0001, beta=0.75, k=1.0)
        )
      )
    )
    (act): NLReLU()
  )
  (seq_blocks): SeqBlocks(
    (blocks): LSTM(128, 64, num_layers=2, batch_first=True)
  )
  (cls_head): Classifier(
    (blocks): ModuleList(
      (0): Sequential(
        (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (layer): Linear(in_features=64, out_features=256, bias=True)
      )
      (1): Sequential(
        (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (layer): Linear(in_features=256, out_features=256, bias=True)
      )
    )
    (act): LeakyReLU(negative_slope=0.01)
    (output_layer): Linear(in_features=256, out_features=13, bias=True)
  )
)

In [70]:
train_loader_length = len(train_loader.dataset)
test_loader_length = len(test_loader.dataset)

def train(model, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # data = transform(data)
        data_length = len(data)
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        del data
        gc.collect()
        torch.cuda.empty_cache()
        loss = F.cross_entropy(output.squeeze(), target)
        del target
        gc.collect()
        torch.cuda.empty_cache()
        optimizer.zero_grad()
        loss.backward()
        # nn.utils.clip_grad.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        # print training stats
        if batch_idx % log_interval == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * data_length}/{train_loader_length} ({100. * batch_idx / train_loader_length:.0f}%)]\tLoss: {loss.item():.6f}")

        # update progress bar
        pbar.update(pbar_update)
        # record loss
        train_losses.append(loss.item())

def test(model, epoch):
    model.eval()
    correct = 0
    for data, target in test_loader:
        # data = transform(data)
        data = data.to(device)
        target = target.to(device)
        output = model(data).squeeze()
        del data
        gc.collect()
        torch.cuda.empty_cache()
        probs = F.softmax(output, dim=-1)
        pred = probs.argmax(dim=-1)
        correct += pred.eq(target).sum().item()
        del target
        gc.collect()
        torch.cuda.empty_cache()
        pbar.update(pbar_update)
    acc = correct / test_loader_length
    print(f"\nTest Epoch: {epoch}\tAccuracy: {correct}/{test_loader_length} ({100. * acc:.0f}%)\n")
    return acc

In [71]:
n_epoch = 10
train_losses = []
test_accuracy = []
optimizer = optim.RAdam(model.parameters(), lr=0.01)
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=0.2,
    steps_per_epoch=len(train_loader), epochs=n_epoch)

In [ ]:
log_interval = 40
pbar_update = 1 / (len(train_loader) + len(test_loader))
model.to(device)
with tqdm(total=n_epoch) as pbar:
    for epoch in range(1, n_epoch + 1):
        train(model, epoch, log_interval)
        acc = test(model, epoch)
        test_accuracy += [acc]
        scheduler.step()

  0%|                                                                           | 0.015873015873015872/10 [00:00<04:11, 25.18s/it]

Train Epoch: 1 [0/8000 (0%)]	Loss: 2.541660


  3%|██▌                                                                         | 0.33333333333333304/10 [00:08<04:12, 26.13s/it]

Train Epoch: 1 [5120/8000 (0%)]	Loss: 2.045665


 10%|███████▋                                                                     | 0.9999999999999973/10 [00:24<03:24, 22.70s/it]

...

 50%|███████████████████████████████████████                                       | 5.015873015872999/10 [02:04<02:00, 24.20s/it]

Train Epoch: 6 [0/8000 (0%)]	Loss: 2.212574


 53%|█████████████████████████████████████████▌                                    | 5.333333333333315/10 [02:13<02:05, 26.79s/it]

Train Epoch: 6 [5120/8000 (0%)]	Loss: 2.129887


 60%|███████████████████████████████████████████████                                | 5.96031746031744/10 [02:28<01:32, 22.83s/it]

In [39]:
n_epoch = 10
n_epoch2 = 50
# train_losses = []
# test_accuracy = []
optimizer = optim.RAdam(model.parameters(), lr=0.01)
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=0.2,
    steps_per_epoch=len(train_loader), epochs=n_epoch2)

log_interval = 40
# pbar_update = 1 / (len(train_loader) + len(test_loader))
# model.to(device)
with tqdm(total=n_epoch2) as pbar:
    for epoch in range(n_epoch+1, n_epoch2 + n_epoch + 1):
        train(model, epoch, log_interval)
        acc = test(model, epoch)
        test_accuracy += [acc]
        scheduler.step()

  0%|                                                                                        | 0.015873015873015872/50 [00:00<17:05, 20.52s/it]

Train Epoch: 11 [0/8000 (0%)]	Loss: 2.102515


  1%|▌                                                                                        | 0.33333333333333304/50 [00:06<16:23, 19.80s/it]

Train Epoch: 11 [5120/8000 (0%)]	Loss: 2.216237


  2%|█▊                                                                                        | 1.0079365079365052/50 [00:19<16:16, 19.93s/it]

...


Test Epoch: 41	Accuracy: 1975/8000 (25%)

Train Epoch: 42 [0/8000 (0%)]	Loss: 2.174847


 62%|████████████████████████████████████████████████████████▋                                  | 31.14285714286042/50 [10:13<06:22, 20.28s/it]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fda41dce100>>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/audio/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
 63%|████████████████████████████████████████████████████████▎                                 | 31.285714285717596/50 [10:16<06:18, 20.21s/it]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fda41dce100>>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/audio/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInte

Train Epoch: 42 [5120/8000 (0%)]	Loss: 2.098204


 63%|████████████████████████████████████████████████████████▋                                 | 31.484126984130338/50 [10:20<06:06, 19.79s/it]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fda41dce100>>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/audio/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
 63%|████████████████████████████████████████████████████████▋                                 | 31.492063492066848/50 [10:20<06:08, 19.91s/it]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fda41dce100>>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/audio/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInte

KeyboardInterrupt: 